In [ ]:
#imports
import glob as glob 
import subprocess
import pathlib
import librosa
import matplotlib.pyplot as plt
import librosa.display
import os 
import shutil
from tqdm.notebook import trange, tqdm
import logging

# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version


In [ ]:
# paths
ksounds_path = '/mnt/bdata/Data/bilal_data/ksounds/'
ksounds_wavpath = '/mnt/bdata/Data/bilal_data/ksounds_wav/'
ksounds_chromepath = '/mnt/bdata/Data/bilal_data/ksounds_chrom/'
ksounds_vid = '/mnt/bdata/Data/bilal_data/mmaction_data/ksounds/'

## Create Ksounds dataset

#### Get Ksounds Video data

In [ ]:
cwd = os.getcwd()

#ksounds_path,ksounds_wavpath,ksounds_chromepath,ksounds_vid

ksvid_train = ksounds_vid+'train/'
ksvid_val = ksounds_vid+'val'
len(glob.glob(ksvid_train+'/*/*.mp4')),len(glob.glob(ksvid_val+'/*/*.mp4'))

ksvid_train = ksounds_wavpath+'train/'
ksvid_val = ksounds_wavpath+'val/'
len(glob.glob(ksvid_train+'/*/*.wav')),len(glob.glob(ksvid_val+'/*/*.wav'))

In [ ]:
ksounds = 'blowing nose, bowling, chopping wood, ripping paper, shuffling cards, singing, tapping pen, typing, blowing out, dribbling ball, laughing, mowing the lawn by pushing lawnmower, shoveling snow, stomping, tap dancing, tapping guitar, tickling, fingerpicking, patting, playing accordion, playing bagpipes, playing bass guitar, playing clarinet, playing drums,playing guitar, playing harmonica, playing keyboard, playing organ, playing piano, playing saxophone, playing trombone, playing trumpet, playing violin, playing xylophone'
ksounds = ksounds.replace(', ',',')
ksounds = ksounds.split(',')
len(ksounds)

In [ ]:
# get list of k400 train files 
k400train_path = '/mnt/bdata/Data/bilal_data/mmaction_data/kinetics400/videos_train'
k400train_files = glob.glob('/mnt/bdata/Data/bilal_data/mmaction_data/kinetics400/videos_train'+'/*/*.mp4')
#print(len(k400train_files))
# get ksounds train folder the destination
ksoundstrain_path = '/mnt/bdata/Data/bilal_data/mmaction_data/ksounds/train'
# loop through k400 train 
classes = []
ctr=0
for file in k400train_files:
    # extract class name 
    class_name = file.split('/')[-2]
    classes.append(class_name)
    #print(class_name)    
    # check if the class name is same as ksounds 

    if class_name in ksounds:
        # if yes : copy that into same folder in ksounds        
        ctr = ctr +1
        # create destination file path         

        dest = file.replace('kinetics400','ksounds')
        dest = dest.replace('videos_train','train')
        print(dest)
        create_path(dest)
        shutil.copy(file,dest)
        logging.info(ctr)

        
    
    # else : just pass
    

    #break

# for classes in ksounds 
# extract wav file
#k400train_files[0]#,ksounds
len(set(classes)),ctr

In [1]:
# for ksounds remove video files that are not in audio 
audio_path = '/mnt/bdata/Data/bilal_data/ksounds_wav/'
files=glob.glob(audio_path+'/*')
print(files)

NameError: name 'glob' is not defined

## Convert mp4 to wav

### Get Ksounds audio dataset

In [ ]:
def create_path(wav_filepath):
    path = wav_filepath.replace(wav_filepath.split('/')[-1],'')
    p = pathlib.Path(path)
    p.mkdir(parents=True, exist_ok=True)
    print(path)
    
def gen_wav(src,dest):
    #print(src)
    command = "ffmpeg -i '"+src+"' -ab 160k -ac 2 -ar 44100 -vn '"+dest+"'"
    subprocess.call(command,shell=True)

def gen_chrom(src,dest):
    x , sr = librosa.load(src,sr=None)
    #print(type(x), type(sr),x.shape, sr)
    #chromagram
    hop_length = 512
    chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length)
    plt.figure(figsize=(5, 5))
    plt.axis('off')
    librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')            
    #print(wav_filepath,chrom_filepath)
    plt.savefig(dest)
    #im = Image.open('test.png')
    #width, height = im.size    
    plt.close('all')

def generate_wav(folder):
    files = glob.glob(folder+'*/*/*.mp4')

    for f in tqdm(files, desc='Generating Wav files'):
        mp4_filepath = f
        wav_filepath = mp4_filepath.split('ksounds')[0]+'ksounds_wav'+mp4_filepath.split('ksounds')[1].split('.')[0]+'.wav'
        chrom_filepath = wav_filepath.split('ksounds_wav')[0]+'ksounds_chrom'+wav_filepath.split('ksounds_wav')[1].split('.')[0]+'.png'   
        #print(mp4_filepath)
        #print(wav_filepath)
        

        create_path(wav_filepath)
        create_path(chrom_filepath)
        gen_wav(mp4_filepath,wav_filepath)
        gen_chrom(wav_filepath,chrom_filepath)   
    #print(len(files))


In [ ]:
generate_wav(ksounds_path)

In [ ]:
mp4_files = glob.glob(ksounds_path+'*/*/*.mp4')
wav_files = glob.glob(ksounds_wavpath+'*/*/*.wav')
chrom_files = glob.glob(ksounds_chromepath+'*/*/*.png')


len(mp4_files), len(wav_files),len(chrom_files)

## generate wav to chromagram

In [ ]:
# wav to png (chromagram)
def generate_chrom(folder):
    files = glob.glob(folder+'*/*/*.wav')
    print(len(files))
    pass
    for audio in tqdm(files,desc='audio to chromagram'):   
        
        wav_filepath = audio        
        chrom_filepath = wav_filepath.split('ksounds_wav')[0]+'ksounds_chrom'+wav_filepath.split('ksounds_wav')[1].split('.')[0]+'.png'    

        x , sr = librosa.load(audio,sr=None)
        #print(type(x), type(sr),x.shape, sr)
        #chromagram
        hop_length = 512
        chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length)
        plt.figure(figsize=(5, 5))
        plt.axis('off')
        librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')        
        create_path(chrom_filepath)
        #print(wav_filepath,chrom_filepath)
        plt.savefig(chrom_filepath)
        #im = Image.open('test.png')
        #width, height = im.size    
        plt.close('all')


In [ ]:
generate_chrom(ksounds_wavpath)    

## Extract Features

In [ ]:
## vid features from TSN


In [ ]:
ksounds_path,ksounds_wavpath,ksounds_chromepath,ksounds_vid

## setup ksounds dataset video

In [ ]:
all = glob.glob(ksounds_vid+'videos/*')
val = glob.glob(ksounds_vid+'val/*/*')



val_files = []
for v in val:
    vname = v.split('/')[-1]      
    print(vname)
    val_files.append(vname)
    break


print(len(val_files))

for f in  all:
    fname = f.split('/')[-1]  
    print(fname)
    if f in val_files:
        print('True')

    break


len(val),len(all)

## Generate TSN compatible files

In [2]:
ksounds

NameError: name 'ksounds' is not defined